In [24]:
import redis
import faker as fk
import pandas
import random
fake = fk.Faker()

# main client
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

In [31]:
def make_rand_computer():
    os = ['windows', 'macOS', 'linux']
    return {"ip": fake.ipv4(), "name": fake.user_name(), "os": random.choice(os)}

def make_rand_mac():
    return fake.mac_address()

def unravel_dict(dictionary):
    return " ".join([f"{key} {value}"for key,value in dictionary.items()])

def add_computer(computer):
    return f"HSET mac:{make_rand_mac()} {unravel_dict(computer)}"

print(add_computer(make_rand_computer()))

HSET mac:0c:99:64:dc:be:25 ip 20.127.34.99 name walkerstephanie os windows


'ip 36.211.164.226 name dking os macOS'